In [1]:
import pandas as pd
import numpy as np
import glob, os, sys
osp = sys.platform

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, TimeDistributed, BatchNormalization
from keras.layers import LSTM, GRU
from keras.optimizers import RMSprop, Adagrad, Adam, SGD
#from keras.models import load_model
import keras.backend as K
# import matplotlib.pyplot as plt
# %matplotlib inline

from utilities import LoadData
from helpers import id_player, id_position, get_player_trajectory

# ---------------------------------------------------------
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
gameid='0021500463'

# directories
# CHANGE HERE
main_dir = '../'
game_dir = main_dir+'data/'
# court_path = data_dir + 'nba_court_T.png'

Data = LoadData(main_dir, game_dir)
data = Data.load_game(gameid)
event_number, moment_number = 0, 0

In [3]:
# get events and number of events
event_df = pd.DataFrame(data['events'])
n_events = event_df.shape[0]
# get numebr of moments
moments = event_df.iloc[event_number,:]['moments']
n_moments = len(moments)

print(n_moments, event_df.shape)
event_df.head()

150 (231, 8)


,end_time_left,home,moments,orig_events,playbyplay,quarter,start_time_left,visitor
0,702.31,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351428029, 708.28, 12.78, None, [[-1,...",[0],GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,708.28,"{'abbreviation': 'TOR', 'players': [{'playerid..."
1,686.28,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351428029, 708.28, 12.78, None, [[-1,...",[1],GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,708.28,"{'abbreviation': 'TOR', 'players': [{'playerid..."
2,668.42,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351444029, 692.25, 12.21, None, [[-1,...","[2, 3]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,692.25,"{'abbreviation': 'TOR', 'players': [{'playerid..."
3,665.71,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351463029, 673.22, 21.44, None, [[-1,...","[4, 5]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,673.22,"{'abbreviation': 'TOR', 'players': [{'playerid..."
4,646.64,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351473027, 668.0, 22.9, None, [[-1, -...",[6],GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,668.00,"{'abbreviation': 'TOR', 'players': [{'playerid..."


In [4]:
# predict next trajectories for single player
pid = 2200
pid_track = pd.DataFrame(event_df['moments'].apply(lambda m: get_player_trajectory(m, pid)))
pid_track = pid_track[pid_track.moments.apply(lambda m: len(m) != 0)]
pid_track.reset_index(inplace=True, drop=True)
pid_track['length'] = pid_track.moments.apply(lambda m: len(m))

In [5]:
pid_track.head()

,moments,length
0,"[[11.15334, 21.35529, 18.38063, 14.07976, 8.56...",150
1,"[[11.15334, 21.35529, 18.38063, 14.07976, 8.56...",550
2,"[[80.35941, 16.31437, 63.70549, 29.93382, 3.52...",596
3,"[[16.77111, 29.51087, 31.06114, 31.5842, 3.586...",289
4,"[[52.14782, 24.2909, 49.76765, 17.82403, 4.968...",610


In [6]:
pid_track.length.sum()/2

32912.5

In [7]:
10%2

0

In [8]:
def chunks(X, length):
    # if odd discard last moment
    len_X = len(X)
    if len_X % 2 != 0:
        len_X -= 1
    return [X[0+i:length+i] for i in range(0, len_X, length)]

train = []
for i in range(len(pid_track)):
    train += chunks(pid_track.moments[i], length=2)

train = np.array(train)

In [9]:
n, m, f = train.shape
train.shape

(32878, 2, 5)

In [10]:
# create label
label = train[:, :, :2].reshape(n*m, 2)
cut = n*m
print(cut)
if label.shape[0] %2 == 0:
    cut -= 1
print(cut, label.shape)
label = label[1:cut, :].reshape((cut-1)//2, 2, 2)
# also remove the last one in train
train = train[:-1, :, :]
train.shape

65756
65755 (65756, 2)


(32877, 2, 5)

In [11]:
# train = train.reshape(-1, 5)
train.shape

(32877, 2, 5)

In [12]:
label.shape

(32877, 2, 2)

In [13]:
# only take the prediction of second timestep
label = label[:, 1, :]

In [14]:
label.shape

(32877, 2)

In [15]:
pid_track.length.min()

6

In [16]:
# # chunk to batch
# # define ftn that generates batches
# def get_minibatches(inputs, targets, batchsize, shuffle=False):
#     assert len(inputs) == len(targets)
#     batches = []
#     target_batches = []
#     if shuffle:
#         indices = np.arange(len(inputs))
#         np.random.shuffle(indices)
#     for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
#         if shuffle:
#             excerpt = indices[start_idx:start_idx + batchsize]
#         else:
#             excerpt = slice(start_idx, start_idx + batchsize)
#         batches.append(inputs[excerpt])
#         target_batches.append(targets[excerpt])
#     return np.array(batches), np.array(target_batches)

# batch_size = 128
# train, label = get_minibatches(train, label, batch_size)

In [17]:
train.shape

(32877, 2, 5)

In [25]:
batch_size = 117
timesteps = 2
input_dim = 5

model = Sequential()
model.add(LSTM(512 ,return_sequences=True, batch_input_shape=(batch_size, timesteps, input_dim), stateful=True))
model.add(LSTM(512 , return_sequences=False, stateful=True))
model.add(Dense (2))
model.add(Activation('linear'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (117, 2, 512)             1060864   
_________________________________________________________________
lstm_5 (LSTM)                (117, 512)                2099200   
_________________________________________________________________
dense_2 (Dense)              (117, 2)                  1026      
_________________________________________________________________
activation_2 (Activation)    (117, 2)                  0         
Total params: 3,161,090
Trainable params: 3,161,090
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.compile(loss='mse', optimizer='adagrad')
#model.compile(loss='mse', optimizer=adagradOpt)
# model.load_weights(model_name)
#model.load_weights('init_weights_minibatch1024_10epochs.h5') # Load the pretrained model
# model.reset_states()

In [30]:
history = model.fit(train, label, epochs = 100, batch_size = batch_size, 
                    verbose = 2, shuffle = True)#, validation_split = 0.1)

Epoch 1/100
 - 2s - loss: 164.8259
Epoch 2/100
 - 2s - loss: 163.9234
Epoch 3/100
 - 2s - loss: 163.0396
Epoch 4/100
 - 2s - loss: 162.1487
Epoch 5/100
 - 2s - loss: 161.2854
Epoch 6/100
 - 2s - loss: 160.3697
Epoch 7/100
 - 2s - loss: 159.5298
Epoch 8/100
 - 2s - loss: 158.6938
Epoch 9/100
 - 2s - loss: 157.8474
Epoch 10/100
 - 2s - loss: 157.0142
Epoch 11/100
 - 2s - loss: 156.1637
Epoch 12/100
 - 2s - loss: 155.3628
Epoch 13/100
 - 2s - loss: 154.5302
Epoch 14/100
 - 2s - loss: 153.7275
Epoch 15/100
 - 2s - loss: 152.9284
Epoch 16/100
 - 2s - loss: 152.1482
Epoch 17/100
 - 2s - loss: 151.3917
Epoch 18/100
 - 2s - loss: 150.6073
Epoch 19/100
 - 2s - loss: 149.8674
Epoch 20/100
 - 2s - loss: 149.0966
Epoch 21/100
 - 2s - loss: 148.3299
Epoch 22/100
 - 2s - loss: 147.6050
Epoch 23/100
 - 2s - loss: 146.8663
Epoch 24/100
 - 2s - loss: 146.1670
Epoch 25/100
 - 2s - loss: 145.4292
Epoch 26/100
 - 2s - loss: 144.6903
Epoch 27/100
 - 2s - loss: 143.9715
Epoch 28/100
 - 2s - loss: 143.2692
E